In [134]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [135]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.3', '2.16.1', '1.5.2', '0.13.0')

In [136]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import backend as k

In [137]:
inicio = time.time()

In [138]:
inicio

1739303024.843334

In [139]:
base = pd.read_csv('C:/Users/nanojau/OneDrive/Documentos/CURSOS/DeepLearningWithPythonUdemy/autos.csv',encoding='ISO-8859-1')
base

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


In [140]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [141]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [142]:
values = {'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}
base = base.fillna(value=values)

In [143]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [144]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [145]:
def criar_rede(loss):
    k.clear_session()
    regressor = Sequential()
    regressor.add(InputLayer(shape=(316,)))
    regressor.add(Dense(units=158, activation='relu', kernel_initializer='random_uniform'))
    regressor.add(Dense(units=158, activation='relu', kernel_initializer='random_uniform'))
    regressor.add(Dense(units=1, activation='linear'))
    regressor.compile(loss=loss, optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [146]:
regressor = KerasRegressor(model=criar_rede, epochs=100, batch_size=300)

In [147]:
parametros = {
    'model__loss': ['mean_squared_error', 'mean_absolute_error']
    # 'model__loss': ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error', 'squared_hinge']
}

In [148]:
grid_search = GridSearchCV(estimator=regressor, param_grid=parametros, cv=3)

In [ ]:
grid_search = grid_search.fit(previsores, preco_real)

Epoch 1/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 75674344.0000 - mean_absolute_error: 4671.5166
Epoch 2/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 53454312.0000 - mean_absolute_error: 3761.4229
Epoch 3/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 46955876.0000 - mean_absolute_error: 3573.4868
Epoch 4/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 41825832.0000 - mean_absolute_error: 3512.0105
Epoch 5/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 37993112.0000 - mean_absolute_error: 3287.6294
Epoch 6/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 40276960.0000 - mean_absolute_error: 3282.3403
Epoch 7/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 36490472.0000 - mean_absolute_error: 3085.2178
Epoch 8/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 38243532.0000 - mean_absolute_error: 3028.1702
Epoch 9/100
799/799 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 39308660.0000 - mean_absolute_error: 2969.4304
Epoch 10/100
799/799 ━━━━━━━

In [ ]:
melhores_parametros = grid_search.best_params_

In [ ]:
melhores_parametros